In [2]:
!pip3 install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.6 MB/s eta 0:00:00
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 3.1 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 23.5 MB/s eta 0:00:000m eta 0:00:01:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 15.7 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 3.8 MB/s eta 0:00:0031m112.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 1.4 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
import pandas as pd
import mlflow
from mlflow.metrics.genai import faithfulness, relevance

In [ ]:
openai_api_key = ""

In [4]:
# Load the data from the CSV file
df = pd.read_csv('deepeval_evaluation.csv', encoding='latin1')

# Extract the relevant columns
inputs = df['input'].tolist()
actual_outputs = df['actual_output'].tolist()
expected_outputs = df['expected_output'].tolist()
retrieval_context = df['retrieval_context'].apply(lambda x: x.split(';') if isinstance(x, str) else [x]).tolist()

In [ ]:
# Create a Pandas DataFrame for evaluation
eval_df = pd.DataFrame({
    "inputs": inputs,
    "outputs": actual_outputs,
    "ground_truth": expected_outputs,
    "context": retrieval_context,  # Rename retrieval_context to context
})

# Convert to an MLflow-compatible dataset
eval_dataset = mlflow.data.from_pandas(
    eval_df, predictions="outputs", targets="ground_truth"
)

# Define relevance and faithfulness metrics
relevance_metric = relevance(model="openai:/gpt-4")
faithfulness_metric = faithfulness(model="openai:/gpt-4")

print(relevance_metric)
print(faithfulness_metric)

In [ ]:
with mlflow.start_run(run_name="RAG_Evaluation"):
    # Run evaluation using the default evaluator and additional metrics
    results = mlflow.evaluate(
        data=eval_dataset,
        model_type="question-answering",
        evaluators=["default"],  # Use default evaluator
        extra_metrics=[relevance_metric, faithfulness_metric],
    )

    # Extract the relevance and faithfulness scores from the results
    relevance_score = results.metrics.get("relevance_metric", None)
    faithfulness_score = results.metrics.get("faithfulness", None)

    # Print the relevance and faithfulness scores
    print(f"Relevance Score: {relevance_score}")
    print(f"Faithfulness Score: {faithfulness_score}")

    # Convert the evaluation results table to a DataFrame
    results_table = results.tables.get("eval_results_table")
    if results_table is not None:
        results_file_path = "rag_evaluation_results.csv"
        results_table.to_csv(results_file_path, index=False)
        print(f"Evaluation results saved to {results_file_path}")
    else:
        print("No results table available.")
